In [1]:
from scipy.optimize import minimize, NonlinearConstraint, basinhopping
import time
import numpy as np

Our additional requirements will be:
- optimise expected_value (prob * risk)
- vector/list of parameters (5 skill entries for each worker -> length = 5*N)
- bound contraints [0,1] for each element
- sum of worker contributions (5 elements) <= how many units they have free to contribute 
- departmental workload is met (This is the hard one because it is dynamic: the constraint function will have to be re-evaluated with each assignment tested by the otpimisation routin. *Can Gekko handle this?* - it appears so from the above, but if not we may need to approximate the workload constraint.)
- add team size constraint

In [2]:
import sys, os
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [3]:
from superscript_model import model
from superscript_model.utilities import Random
from superscript_model.project import Project
from superscript_model.organisation import Team
from superscript_model.config import MIN_TEAM_SIZE, MAX_TEAM_SIZE

In [4]:
STEPS = 250

abm = model.SuperScriptModel(worker_count=60, 
                             new_projects_per_timestep=2, 
                             worker_strategy = "Stake",
                             organisation_strategy = 'Basic')

In [5]:
abm.run_model(STEPS)

In [6]:
tracked = abm.datacollector.get_model_vars_dataframe()
tracked.head(STEPS)

,ActiveProjects,RecentSuccessRate,SuccessfulProjects,FailedProjects,NullProjects,WorkersOnProjects,WorkersWithoutProjects,WorkersOnTraining,AverageTeamSize,AverageSuccessProbability,AverageWorkerOvr,AverageTeamOvr,WorkerTurnover,ProjectLoad,TrainingLoad,DeptLoad,Slack,ProjectsPerWorker
0,2,0.000000,0.0,0.0,0,10,50,0,7.000000,0.086290,50.720044,61.266602,0.0,0.103333,0.000000,0.1,0.796667,0.233333
1,4,0.000000,0.0,0.0,0,12,48,0,7.000000,0.043145,50.336153,52.231332,0.0,0.173333,0.000000,0.1,0.726667,0.466667
2,6,0.000000,0.0,0.0,0,21,39,0,6.500000,0.036095,50.046738,48.892455,0.0,0.210000,0.000000,0.1,0.690000,0.650000
3,7,0.000000,0.0,0.0,0,23,37,0,6.571429,0.045526,49.776247,49.624497,0.0,0.251667,0.000000,0.1,0.648333,0.766667
4,8,0.000000,0.0,2.0,0,27,33,0,6.625000,0.115061,49.384737,50.969606,0.0,0.240000,0.000000,0.1,0.660000,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,9,0.041667,0.0,2.0,0,22,31,7,6.000000,0.164353,57.292271,58.615080,3.0,0.231667,0.116667,0.1,0.551667,0.716667
246,10,0.075000,1.0,0.0,0,24,29,7,6.000000,0.115762,58.469073,57.190034,2.0,0.235000,0.116667,0.1,0.548333,0.816667
247,8,0.072222,0.0,3.0,0,20,32,8,6.250000,0.134427,59.751554,60.342500,6.0,0.203333,0.133333,0.1,0.563333,0.650000
248,8,0.059722,0.0,1.0,1,21,34,5,6.250000,0.133046,59.696760,58.326247,9.0,0.196667,0.083333,0.1,0.620000,0.650000


#### Now, with the current state of the system, we will try creating a new project and finding its optimal team allocation:

In [7]:
project = Project(abm.inventory, 
                  abm.inventory.index_total + 1,
                  project_length=5,
                  start_time_offset=abm.inventory.get_start_time_offset(),
                  start_time=abm.schedule.steps)

In [8]:
print(project.requirements.to_string())

{
    "risk": 5,
    "creativity": 5,
    "flexible_budget": false,
    "budget": 19,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 15,
    "hard_skills": {
        "A": {
            "level": 2,
            "units": 4
        },
        "B": {
            "level": 1,
            "units": 3
        },
        "C": {
            "level": 1,
            "units": 5
        },
        "D": {
            "level": null,
            "units": 0
        },
        "E": {
            "level": 1,
            "units": 3
        }
    }
}


In [9]:
bid_pool = abm.inventory.team_allocator.strategy.invite_bids(project)
print(len(bid_pool))

49


In [10]:
# for m in bid_pool:
#      if m.skills.hard_skills['C'] > 4:
#         print(m.worker_id)

for m in bid_pool:
    for skill in ['A', 'B', 'C', 'D', 'E']:
         if m.skills.hard_skills[skill] > 5:
            print(skill, m.worker_id)

In [11]:
x = np.zeros(len(bid_pool) * 5)

x0 = np.zeros(len(x))
bounds = list([(0,1) for i in x])

In [12]:
print(MAX_TEAM_SIZE) # = 7
print(MIN_TEAM_SIZE) # = 3

7
3


#### We now add the constraint on the number uf units that each worker can contribute to the project (worker_unit_budget):

In [13]:
# project_1 = project
# project = project_2

In [14]:
worker_unit_budgets = {
    worker.worker_id: worker.contributions.get_remaining_units(
        project.start_time, project.length
    )
    for worker in bid_pool
}
print(min(worker_unit_budgets.values()))
print(max(worker_unit_budgets.values()))

worker_ids = list(worker_unit_budgets.keys())
worker_unit_budgets = worker_unit_budgets

2
10


In [118]:
worker_unit_budgets

{1: 10,
 3: 7,
 5: 6,
 7: 3,
 8: 7,
 17: 10,
 19: 10,
 21: 10,
 26: 8,
 27: 6,
 29: 10,
 42: 4,
 44: 7,
 45: 10,
 47: 7,
 48: 5,
 53: 10,
 57: 8,
 59: 10,
 64: 10,
 65: 6,
 66: 10,
 67: 6,
 69: 10,
 74: 10,
 75: 10,
 76: 2,
 77: 6,
 78: 8,
 80: 7,
 81: 10,
 82: 10,
 83: 7,
 87: 10,
 89: 7,
 93: 6,
 95: 10,
 96: 10,
 97: 10}

In [119]:
constraints = []
constraints_slsqp = []

for wi, worker in enumerate(bid_pool):
    start = wi*5
    constraints.append(
        NonlinearConstraint(lambda x: sum(np.round(x[start:start+5])) - worker_unit_budgets[worker.worker_id], -np.inf, 0)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: worker_unit_budgets[worker.worker_id] - sum(np.round(x[start:start+5])),
        'name': 'worker_unit_budget_%d' % worker.worker_id
    })

#### We now add the deparmental workload constraint, which is dynamic and changes with the assignments in the 'x' vector: 

In [120]:
base_dept_unit_budgets = {
            dept.dept_id: dept.get_remaining_unit_budget(
                project.start_time, project.length
            )
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_ids = base_dept_unit_budgets.keys()
base_dept_unit_budgets = base_dept_unit_budgets

In [121]:
dept_members = {
            dept.dept_id: [m for m in bid_pool if m.department.dept_id==dept.dept_id]
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_members = dept_members

In [122]:
base_dept_unit_budgets

{7: 33.0,
 8: 32.0,
 9: 18.0,
 0: 32.0,
 1: 22.0,
 2: 36.0,
 3: 38.0,
 4: 30.0,
 5: 33.0,
 6: 49.0}

In [123]:
def adjusted_dept_unit_budget(x, dept_id):
    
    base = base_dept_unit_budgets[dept_id]
    for worker in dept_members[dept_id]:
        start = bid_pool.index(worker)
        base -= sum(np.round(x[start:start+5]))
        
    return base

In [124]:
for dept_id in dept_ids:
    constraints.append(
        NonlinearConstraint(lambda x: adjusted_dept_unit_budget(x, dept_id), 0, np.inf)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: adjusted_dept_unit_budget(x, dept_id),
        'name': 'dept_budget_%d' % dept_id
    })

#### Now we add the team size constraint:

In [125]:
def team_size(x):
    
    size = 0
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        if sum(np.round(x[start:start+5])) > 0:
            size += 1
            
    return size

In [126]:
constraints.append(NonlinearConstraint(team_size, MIN_TEAM_SIZE, MAX_TEAM_SIZE))
#constraints_slsqp = []

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: MAX_TEAM_SIZE - team_size(x), 'name': 'team_size_ub'
})
constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: team_size(x) - MIN_TEAM_SIZE, 'name': 'team_size_lb'
})

#### Add bound constraints (COBYLA only):

In [127]:
#constraints_slsqp = []

for i, xi in enumerate(x):
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: x[i], 'name': 'lb_%d' % i
    })
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: 1 - x[i] , 'name': 'ub_%d' % i
    })

#### And team budget constraint:

In [128]:
constraints.append(NonlinearConstraint(int(get_team(x).within_budget()), 1, np.inf))
#constraints_slsqp = []

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: -1 + int(get_team(x).within_budget()), 'name': 'budget_constraint'
})


In [25]:
constraints = tuple(constraints)

#### Now we define the objective function:

#### Note: we use 0.5 as the boundary to indiciate contributions is non-zero

In [26]:
## does bid pool need to be a m.Param?
## does project need to be a m.Param?

def get_team(x):
    
    SKILLS = ['A', 'B', 'C', 'D', 'E']
    contributions = dict()
    #for skill in project.required_skills:
    for skill in SKILLS:
        contributions[skill] = []
    
    team_members = {}
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        in_team = False
        
        #for si, skill in enumerate(project.required_skills):
        for si, skill in enumerate(SKILLS):
            if x[start+si] > 0.5:
                contributions[skill].append(worker_id)
                in_team = True
        
        if in_team:
            team_members[worker_id] = bid_pool[wi]
    
    if len(team_members) == 0:
        return Team(project, {}, None)
    else:
        return Team(project, 
                    team_members, 
                    team_members[list(team_members.keys())[0]],
                    contributions=contributions)    

In [27]:
def objective_func(x):
    
    test_team = get_team(x)
    
    #assert test_team.within_budget()
    
    project.team = test_team
    abm.inventory.success_calculator.calculate_success_probability(project)
    
    return -project.success_probability

# TODO:

- add constraint that unrequired hard skills are equal to zero.
- add project budget constraint (and do: assert test_team.within_budget())
- (assign worker contributions in Team! (get_team?))
- test max_team_sze. Increase? Specify equality and check it works - inspect guesses manually

Note: COBYLA ran 'successfully' (before changing 'contributions=contributions' when calling Team()) with maxiter=1000 and ftol=1e-08. But only assigned three elements as non-zero for a project with 6 reuqired skilll units. 

After changing the above, COBYLA also runs 'successfully'...('result')  

'trust-constr' still gives "LinAlgError: SVD did not converge"

'SLSQP' with bespoke bounds (correct?) currently giving exit mode 8 or 5 depending on initial guess.

'COBYLA' finding a solution! But violating constraints so not optimal! Specify bounds as constraints...

#### Note: one option to try is just letting the x vector define team membership (and revert to automatic skill allocation). Then vector is length N.

In [28]:
guess_team = abm.inventory.team_allocator.strategy.select_team(project, bid_pool)

In [29]:
print(guess_team.to_string())

{
    "project": 51,
    "members": [
        66,
        53,
        74,
        69,
        89
    ],
    "lead": 89,
    "success_probability": 0.0,
    "team_ovr": 38.5,
    "skill_balance": 7.4,
    "creativity_match": 3.7,
    "skill_contributions": {
        "B": [
            74,
            66,
            53,
            69
        ],
        "C": [
            69,
            53,
            66,
            74
        ],
        "D": [
            66,
            74,
            89,
            69,
            53
        ],
        "E": [
            89,
            74,
            53,
            66,
            69
        ]
    }
}


In [30]:
def get_x(team):
    
    x = np.zeros(len(bid_pool*5))
    for member_id in team.members:
        start = 5*[m.worker_id for m in bid_pool].index(member_id)
        for si, skill in enumerate(['A', 'B', 'C', 'D', 'E']):
            if skill in project.required_skills:
                x[start+si] = 1
                
    return x

In [647]:
method = 'COBYLA' #'COBYLA' #'SLSQP' # 'trust-constr'
maxiter = 500
ftol = 1e-09
eps=1e-8
catol = 0.0
tol=1e-2
rhobeg = 0.6
verbose=3
factorization_method='QRFactorization'
initial_tr_radius=100

#guess=list([Random.randint(0,1) for i in range(len(x))])
guess=np.zeros(len(x))
#guess=np.ones(len(x))
#guess[1] = 1.0
#guess[2] = 1.0
#guess[3] = 1.0
#guess[4] = 1.0
#guess[6] = 1.0
#guess[7] = 1.0
#guess = get_x(guess_team)

# For trust-constr: (LinAlgError: SVD did not converge)
# result_1 = minimize(objective_func, guess, method=method, bounds=bounds, constraints=constraints,
#                 options={'maxiter': maxiter, 'verbose': verbose, 
#                          'factorization_method': factorization_method,
#                          'initial_tr_radius': initial_tr_radius})#,

# For COBYLA:
result_1 = minimize(objective_func, guess, method=method,
                    constraints=constraints_slsqp, options={'maxiter': maxiter, 'disp': True,
                                                            'catol': catol, 'rhobeg': rhobeg})

In [649]:
print(team_size(result_1.x))
print(result_1)
print(get_team(result_1.x).to_string())

2
     fun: -0.6278800936309762
   maxcv: 3.0
 message: 'Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.'
    nfev: 500
  status: 4
 success: False
       x: array([-1.82709926e-01, -3.99293742e-01,  2.92600518e-01, -1.35338602e-02,
       -2.11256213e-02,  2.55162711e-02, -3.24962972e-02,  1.39230407e-02,
       -2.09356767e-02,  1.07983884e-02,  1.93948636e-02, -1.58079441e-02,
       -4.28437204e-02, -1.71077332e-02, -1.11319542e-02,  3.80565941e-02,
        6.51166759e-01,  6.11852629e-01, -2.60091992e-02, -2.34817381e-02,
        1.62255801e-02,  2.61270921e-02, -1.25813563e-02,  9.19583094e-04,
        1.43394997e-02,  3.49713043e-02,  6.49660077e-01, -1.66849676e-02,
       -2.47610223e-02, -1.71531578e-02,  2.72360393e-03, -1.76515150e-02,
       -1.72868578e-02, -3.56823421e-03, -7.52398655e-03, -5.05397778e-03,
       -1.96877696e-02, -1.85435242e-02, -2.88968215e-02, -2.62287064e-03,
       -7.07644898e-04, -4.36375985e-03, 

#### Trying basinhopping with COBYLA at each step...

Notes:
- this keeps finding team 1,5,9 with p=0.603
- with niter=1000 it found 5,9,16,31 with p=0.604, took209 seconds
- with niter=10000 it found 1,5,9 with p=0.610, took 2345 seconds (allocation: "B": [5,9],"C": [1,5], "D": [1], "E": [1]
- on second project. with niter=1000, took 629 seconds to find 1,5,6,9. Why same workers?! p=0.647
- on second project. with niter=100, took 67 seconds to find 5,7,63. with p=0.670
- on second project. with niter=100, took 67 seconds to find 1,9,36. with p=0.637
- (attempt 5) third project. with niter=100, took 57 seconds to find 1,3,5,7,17,42,45. with p=0.837
- (attempt 6) third project. with niter=100, took 51 seconds to find 1,3,5,7,17,42,45. with p=0.920 (different contributions)
- (attempt 7) third project. with niter=100, took 47 seconds to find  1,3,5,7,42,59,93. with p=0.72
- (attempt 8) third project. with niter=1000, took 515 seconds to find 1,3,5,7,42,45,65. with p=0.834 
 
- not sure that COBYLA is doing much at each hop?
- try seeding random number generator in SS.utilties for reproducible results
- the returned value does meet constraints, even if COBYLA can't find a local minimum that meets constraints. 

In [129]:
def test_constraints(x):
    
    for cons in constraints_slsqp:
        assert cons['type'] == 'ineq'
        if cons['fun'](x) < 0:
            print("Constraint violated: %s" % cons['name'])
            return False
        
    return True

In [130]:
class MyConstraints(object):
    
    def __init__(self):
        self.test = test_constraints
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        return self.test(x)

#### Define bespoke step taking:

(Note: without this the basinhopping algorithm perturbs all vector elements by a random amount and result in the team size ub constraint being violated all the time.)

This step function could be made smarter, e.g. by focusing only on required skills. And by choosing the workers to add/remove based on some hueristic rather than randomly...

In [132]:
class MyTakeStep(object):
    
    def __init__(self, bid_pool=bid_pool): #, stepsize=0.5):
        #self.stepsize = stepsize
        self.name = 'add/remove workers'
        self.bid_pool = bid_pool
   
    def __call__(self, x):
        
        # determine how many workers to add and remove to team:
        number_to_add = min(Random.randint(0, MAX_TEAM_SIZE), 
                            len(bid_pool) - team_size(x))
        
        new_size = team_size(x) + number_to_add
        min_remove = max(0, new_size - MAX_TEAM_SIZE)
        max_remove = new_size - MIN_TEAM_SIZE
        if max_remove < min_remove:
            number_to_remove = 0
        else:
            number_to_remove = Random.randint(min_remove, max_remove)
        
        # choose members to add:
        assert len(bid_pool) >= number_to_add + number_to_remove
        current_team = get_team(x)
        
        to_add = []
        new_team_members = list(current_team.members.values())
        inverted = bid_pool[::-1]
        for i in range(number_to_add):
            chosen = Random.choice(inverted) 
            while chosen in new_team_members:
                chosen = Random.choice(inverted) 
            new_team_members.append(chosen)
            to_add.append(chosen)
        
        for a in to_add:
            start = bid_pool.index(a) * 5
            add_skills = Random.choices(range(5), 
                           min(5, worker_unit_budgets[a.worker_id]))
            for si in add_skills:
                x[start+si] = 1
        
        # now select and remove required numer of workers:
        to_remove = Random.choices(new_team_members, number_to_remove)
        for r in to_remove:
            start = bid_pool.index(r) * 5
            for i in range(5):
                x[start+i] = 0
        
        return x

In [133]:
my_constraints = MyConstraints()
my_takestep = MyTakeStep()

maxiter = 100
catol = 0.0
#tol=1e-2
rhobeg = 0.6
niter = 1000

guess=np.zeros(len(x))

minimizer_kwargs = {"method":'COBYLA',
                    'constraints': constraints_slsqp, 
                    'options': {'maxiter': maxiter, 'disp': True, 
                                'catol': catol, 'rhobeg': rhobeg}}

def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))
    if accepted:
        print(team_size(x))

In [134]:
start_time = time.time()

ret = basinhopping(objective_func, guess, 
                   minimizer_kwargs=minimizer_kwargs,
                   niter=niter, callback=print_fun, seed=70470, 
                   accept_test=my_constraints,
                   take_step=my_takestep)

elapsed_time = time.time() - start_time  
print("%d iterations took %.2f seconds" % (niter, elapsed_time))

Constraint violated: team_size_ub
at minimum -0.6718 accepted 0
Constraint violated: team_size_ub
at minimum -0.2021 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: team_size_ub
at minimum -0.4456 accepted 0
Constraint violated: team_size_ub
at minimum -0.2986 accepted 0
Constraint violated: budget_constraint
at minimum -0.6289 accepted 0
Constraint violated: team_size_ub
at minimum -0.5171 accepted 0
Constraint violated: team_size_ub
at minimum -0.5570 accepted 0
Constraint violated: budget_constraint
at minimum -0.6837 accepted 0
Constraint violated: team_size_ub
at minimum -0.2204 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.2709 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: budget_constraint
at minimum -0.4663 accepted 0
Constraint violated: budget_constraint
at minimum -0.7909 accepted 0
Const

Constraint violated: team_size_ub
at minimum -0.6385 accepted 0
Constraint violated: team_size_ub
at minimum -0.6430 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: budget_constraint
at minimum -0.6080 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.6620 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.3362 accepted 0
Constraint violated: budget_constraint
at minimum -0.5109 accepted 0
Constraint violated: budget_constraint
at minimum -0.5111 accepted 0
Constraint violated: budget_constraint
at minimum -0.4587 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: budget_constraint
at minimum -0.8

Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.3605 accepted 0
Constraint violated: team_size_ub
at minimum -0.7001 accepted 0
Constraint violated: team_size_ub
at minimum -0.7284 accepted 0
Constraint violated: team_size_ub
at minimum -0.3649 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: team_size_ub
at minimum -0.5627 accepted 0
Constraint violated: budget_constraint
at minimum -0.6526 accepted 0
Constraint violated: budget_constraint
at minimum -0.5193 accepted 0
Constraint violated: team_size_ub
at minimum -0.3522 accepted 0
Constraint violated: team_size_ub
at minimum -0.2466 accepted 0
Constraint violated: team_size_ub
at minimum -0.5115 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: team_size_ub
at minimum -0.2466 accepted 0
Constraint violated: budget_constraint
at minimum -0.6470 accepted 0
Constraint

Constraint violated: budget_constraint
at minimum -0.6381 accepted 0
Constraint violated: team_size_ub
at minimum -0.6442 accepted 0
Constraint violated: team_size_ub
at minimum -0.2057 accepted 0
Constraint violated: budget_constraint
at minimum -0.7216 accepted 0
Constraint violated: budget_constraint
at minimum -0.6218 accepted 0
Constraint violated: team_size_ub
at minimum -0.3668 accepted 0
Constraint violated: team_size_ub
at minimum -0.7138 accepted 0
Constraint violated: team_size_ub
at minimum -0.6585 accepted 0
Constraint violated: team_size_ub
at minimum -0.6958 accepted 0
Constraint violated: team_size_ub
at minimum -0.3644 accepted 0
Constraint violated: budget_constraint
at minimum -0.5509 accepted 0
Constraint violated: team_size_ub
at minimum -0.4514 accepted 0
Constraint violated: team_size_ub
at minimum -0.5992 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.3622 accepted 0
Constraint viol

Constraint violated: team_size_ub
at minimum -0.2905 accepted 0
Constraint violated: budget_constraint
at minimum -0.5399 accepted 0
Constraint violated: team_size_ub
at minimum -0.6281 accepted 0
Constraint violated: team_size_ub
at minimum -0.4525 accepted 0
Constraint violated: budget_constraint
at minimum -0.5593 accepted 0
Constraint violated: team_size_ub
at minimum -0.6421 accepted 0
Constraint violated: budget_constraint
at minimum -0.9200 accepted 0
Constraint violated: budget_constraint
at minimum -0.4460 accepted 0
Constraint violated: team_size_ub
at minimum -0.2421 accepted 0
Constraint violated: budget_constraint
at minimum -0.5207 accepted 0
Constraint violated: budget_constraint
at minimum -0.4502 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: team_size_ub
at minimum -0.5109 accepted 0
Constraint violated: budget_constraint
at minimum -0.5315 accepted 0
Constraint violated: team_size_ub
at minimum -0.3391 accepted 0


Constraint violated: budget_constraint
at minimum -0.8369 accepted 0
Constraint violated: team_size_ub
at minimum -0.6297 accepted 0
Constraint violated: team_size_ub
at minimum -0.2439 accepted 0
Constraint violated: budget_constraint
at minimum -0.5264 accepted 0
Constraint violated: team_size_ub
at minimum -0.5960 accepted 0
Constraint violated: team_size_ub
at minimum -0.7462 accepted 0
Constraint violated: team_size_ub
at minimum -0.8593 accepted 0
Constraint violated: budget_constraint
at minimum -0.7144 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: team_size_ub
at minimum -0.3089 accepted 0
Constraint violated: team_size_ub
at minimum -0.5595 accepted 0
Constraint violated: budget_constraint
at minimum -0.5603 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.4423 accepted 0
Constraint violated: budget_constraint
at minimum -0.5331 accepted 0
Const

Constraint violated: team_size_ub
at minimum -0.1905 accepted 0
Constraint violated: team_size_ub
at minimum -0.5465 accepted 0
Constraint violated: team_size_ub
at minimum -0.5769 accepted 0
Constraint violated: budget_constraint
at minimum -0.8869 accepted 0
Constraint violated: team_size_ub
at minimum -0.1730 accepted 0
Constraint violated: team_size_ub
at minimum -0.7185 accepted 0
Constraint violated: team_size_ub
at minimum -0.6981 accepted 0
Constraint violated: budget_constraint
at minimum -0.5951 accepted 0
Constraint violated: budget_constraint
at minimum -0.5597 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.6385 accepted 0
Constraint violated: budget_constraint
at minimum -0.6395 accepted 0
Constraint violated: budget_constraint
at minimum -0.4654 accepted 0
Constraint violated: team_size_ub
at minimum -0.8351 accepted 0
Constraint violated: team_size_ub
at minimum -0.9517 accepted 0
Constraint

Constraint violated: team_size_ub
at minimum -0.6246 accepted 0
Constraint violated: budget_constraint
at minimum -0.5256 accepted 0
Constraint violated: team_size_ub
at minimum -0.6124 accepted 0
Constraint violated: team_size_ub
at minimum -0.6309 accepted 0
Constraint violated: team_size_ub
at minimum -0.3273 accepted 0
at minimum 0.0000 accepted 0
Constraint violated: team_size_ub
at minimum -0.7383 accepted 0
Constraint violated: team_size_ub
at minimum -0.2959 accepted 0
Constraint violated: budget_constraint
at minimum -0.8020 accepted 0
Constraint violated: budget_constraint
at minimum -0.2164 accepted 0
Constraint violated: team_size_ub
at minimum -0.4654 accepted 0
Constraint violated: budget_constraint
at minimum -0.4511 accepted 0
Constraint violated: team_size_ub
at minimum -0.6289 accepted 0
Constraint violated: team_size_ub
at minimum -0.5272 accepted 0
Constraint violated: budget_constraint
at minimum -0.4218 accepted 0
Constraint violated: team_size_ub
at minimum -0.24

Constraint violated: budget_constraint
at minimum -0.5689 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: team_size_ub
at minimum -0.7097 accepted 0
Constraint violated: team_size_ub
at minimum -0.6439 accepted 0
Constraint violated: budget_constraint
at minimum -0.4789 accepted 0
Constraint violated: budget_constraint
at minimum -0.6971 accepted 0
Constraint violated: budget_constraint
at minimum -0.8869 accepted 0
Constraint violated: budget_constraint
at minimum -0.5714 accepted 0
Constraint violated: budget_constraint
at minimum -0.7525 accepted 0
Constraint violated: budget_constraint
at minimum -0.8369 accepted 0
Constraint violated: team_size_ub
at minimum -0.7551 accepted 0
1000 iterations took 557.20 seconds


In [135]:
ret

                        fun: -0.7525352016417917
 lowest_optimization_result:      fun: -0.7525352016417917
   maxcv: 1.0
 message: 'Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.'
    nfev: 100
  status: 4
 success: False
       x: array([0. , 0. , 0.6, 0. , 0. , 0.6, 0. , 0. , 0.6, 0. , 0.6, 0.6, 0. ,
       0. , 0.6, 0.6, 0. , 0. , 0. , 0. , 0.6, 0. , 0. , 0. , 0. , 0.6,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0.6, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 

In [100]:
#objective_func(result_1.x)
objective_func(ret.x)

-0.8020162742065761

In [101]:
best_team = get_team(ret.x)
project.team = best_team
abm.inventory.success_calculator.calculate_success_probability(project)

In [102]:
print(best_team.to_string())

{
    "project": 51,
    "members": [
        1,
        3,
        5,
        7,
        8,
        17,
        42
    ],
    "lead": 1,
    "success_probability": 0.8,
    "team_ovr": 119.2,
    "skill_balance": 1.4,
    "creativity_match": 0.2,
    "skill_contributions": {
        "A": [
            3,
            5,
            7,
            8,
            17
        ],
        "B": [
            5
        ],
        "C": [
            1
        ],
        "D": [
            42
        ],
        "E": [
            5
        ]
    }
}


In [149]:
best_team.skill_balance

1.4315207243749024

In [158]:
test = [1,2,3,0,5]
test.sort(reverse=True)
test[0:10]

TypeError: 'NoneType' object is not subscriptable

In [145]:
print(bid_pool[worker_ids.index(42)].skills.to_string())

{
    "Worker OVR": 66.7,
    "Hard skills": [
        2.2,
        3.0,
        4.2,
        6.9,
        0.5
    ],
    "Soft skills": [
        2.4,
        3.7,
        3.3,
        2.7,
        2.0
    ],
    "Hard skill probability": 0.8,
    "OVR multiplier": 20
}


In [148]:
print(project.requirements.to_string())

{
    "risk": 25,
    "creativity": 5,
    "flexible_budget": true,
    "budget": 38.75,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 19,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 3,
            "units": 4
        },
        "C": {
            "level": 2,
            "units": 4
        },
        "D": {
            "level": 1,
            "units": 6
        },
        "E": {
            "level": 1,
            "units": 5
        }
    }
}


In [73]:
attempt = 9

In [74]:
import pickle
with open('best_team_attempt_%d.json' % attempt, 'wb') as ofile:
    pickle.dump(best_team.to_string(), ofile)

In [75]:
with open('bid_pool_attempt_%d.json' % attempt, 'wb') as ofile:
    pickle.dump(bid_pool, ofile)

In [76]:
with open('project_attempt_%d.json' % attempt, 'wb') as ofile:
    pickle.dump(project, ofile)

*attempt_1 (project 1, niter=10000)
* attempt_2  (project 2, niter=1000)
* [attempt_3  (project 2, niter=100)]
* attempt_4  (project 2, niter=100)
* attempt_5  (project 3, niter=100) *New system state. ABM recreated and run for 25 steps. Bid pool of 39.
* attempt_6  (project 3, niter=100). Same team as attempt 5!
* attempt_7 (projet 3, niter=100). Worse.


#### Note that our methods seems to favour lower numbers (1,5,7,9)? Tried new abm to determine if this was to do with specific workers...

In [704]:
best_team.within_budget()

False

In [93]:
print(project.requirements.to_string())

{
    "risk": 5,
    "creativity": 1,
    "flexible_budget": false,
    "budget": 25,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 6,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 5,
            "units": 2
        },
        "C": {
            "level": 5,
            "units": 2
        },
        "D": {
            "level": 2,
            "units": 1
        },
        "E": {
            "level": 3,
            "units": 1
        }
    }
}


In [97]:
sum(result.x)

4.016970683373013

In [98]:
objective_func(result.x)

-0.5078404640292967

In [99]:
best_team = get_team(result.x)
project.team = best_team
abm.inventory.success_calculator.calculate_success_probability(project)

In [100]:
print(best_team.to_string())

{
    "project": 31,
    "members": [
        5,
        9,
        45
    ],
    "lead": 5,
    "success_probability": 0.5,
    "team_ovr": 84.1,
    "skill_balance": 5.5,
    "creativity_match": 0.4,
    "skill_contributions": {
        "A": [
            45
        ],
        "B": [
            5,
            9
        ],
        "C": [
            5
        ],
        "D": [],
        "E": []
    }
}


In [110]:
with open('project_attempt_6.json', 'rb') as ifile:
    temp = pickle.load(ifile)
    
print(temp.requirements.to_string())

{
    "risk": 25,
    "creativity": 5,
    "flexible_budget": true,
    "budget": 38.75,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 19,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 3,
            "units": 4
        },
        "C": {
            "level": 2,
            "units": 4
        },
        "D": {
            "level": 1,
            "units": 6
        },
        "E": {
            "level": 1,
            "units": 5
        }
    }
}


In [112]:
with open('best_team_attempt_6.json', 'rb') as ifile:
    temp = pickle.load(ifile)
    
print(temp)

{
    "project": 51,
    "members": [
        1,
        3,
        5,
        7,
        17,
        42,
        45
    ],
    "lead": 1,
    "success_probability": 0.9,
    "team_ovr": 127.6,
    "skill_balance": 0.9,
    "creativity_match": 0.1,
    "skill_contributions": {
        "A": [
            1,
            3,
            5,
            7,
            17
        ],
        "B": [
            5
        ],
        "C": [
            45
        ],
        "D": [
            3,
            42
        ],
        "E": [
            5
        ]
    }
}


In [84]:
type(best_team.project.success_probability)

numpy.float64

In [86]:
best_team.round_to

{'A': [1, 5, 6], 'B': [], 'C': [], 'D': [], 'E': []}